! **WORK IN PROGRESS** Could still fail without reason !

This is the Open source of the paper <AnimeGAN: a novel lightweight GAN for photo animation>, which uses the GAN framwork to transform real-world photos into anime images.

In [ ]:
# Input image that will be converted to anime style
file_image = '' #@param {type: "string"}

# Anime Style
image_model = "Hayao"  #@param ["Hayao", "Shinkai", "Paprika"]

output_path = "/content/output"


In [ ]:
import mimetypes
mimetypes.init()

mimestart = mimetypes.guess_type(file_image)[0]
mimestart = mimestart.split('/')[0]
is_video = mimestart == 'video'

input_path = "/tmp/sr_input"
!rm -rf $input_path
!mkdir -p $input_path

# Copy image or extract video frames and copy to input_path

if is_video:
    # Save video framerate
    framerate = !ffprobe -v error -select_streams v:0 -show_entries stream=r_frame_rate -of default=noprint_wrappers=1:nokey=1 $file_image
    framerate = float(framerate[0].split('/')[0])
    print("framerate", framerate)

    # Extract frames
    !ffmpeg -i $file_image $input_path/%05d.jpg
else:
    !cp $file_image $input_path/


### STAGE ONE: Prepare dependencies.

In [ ]:
%tensorflow_version 1.x
!ls -l /usr/local/cuda
!git clone https://github.com/CyFeng16/MVIMP.git

import os
os.chdir('/content/MVIMP')
!python3 preparation.py

### STAGE TWO: **Upload** your pictures in `MVIMP/Data/Input`.<br>Processed outputs will be in `MVIMP/Data/Output`.

### STAGE THREE: One-line commmand to embrace AnimeGANv2(change your photos' style to anime-like).

| params 	| abbr. 	| Default 	| Description 	|
|-	|-	|-	|-	|
| --style 	| -s 	| Hayao 	| The anime style you want to get. 	|

| Style name 	| Anime style 	|
|-	|-	|
| Hayao 	| Miyazaki Hayao 	|
| Shinkai 	| Makoto Shinkai 	|
| Paprika 	| Kon Satoshi 	|

In [ ]:
!mkdir -p /content/MVIMP/Data/Input

!apt-get install imagemagick
print("running convert")
!convert {file_image} /content/MVIMP/Data/Input/input.jpg

!python3 inference_animeganv2.py -s {image_model}
!cp -v /content/MVIMP/Data/Output/* {output_path}

In [ ]:

if is_video:
    # Create video
    !ffmpeg -framerate $framerate -i {output_path}/%*.jpg -c:v libx264 -r 30 -pix_fmt yuv420p $input_path/output.mp4
    
    # Check if original video contains audio stream
    has_audio_stream = !ffprobe -v error -select_streams a -show_entries stream=codec_name -of default=noprint_wrappers=1:nokey=1 $video_file
    print("has_audio_stream",has_audio_stream)
    has_audio_stream = len(has_audio_stream) > 0

    if has_audio_stream:
        # Combine with audio of original video
        !ffmpeg -i $input_path/output.mp4 -i $video_file -c copy -map 0:v -map 1:a -shortest $output_path/output.mp4
    else:
        # Copy output video
        !cp $input_path/output.mp4 $output_path/output.mp4

    !rm {output_path}/*.jpg